In [1]:
import os
import json
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from transformers import BertTokenizer, BertModel
import cv2
from tqdm import tqdm
from skimage.metrics import structural_similarity as ssim
import csv

d:\Github\ICRA_25_workshop\.nsai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class RH20TDataset():
    def __init__(self, data_path):
        self.data_path = data_path
        self.depth_path = r"C:\Users\azahid\Downloads\RH20T_cfg2_depth\RH20T_cfg2"
        self.root_dir_output = r'D:\Github\ICRA_25_workshop\dataset_preparation\dataset_depth_cameras_850_215'
        self.camera = ['cam_104122061850', 'cam_036422060215']
        # self.camera = ['cam_f0461559', 'cam_105422061350', 'cam_104422070044', 'cam_104422070042', 'cam_104122063678', 'cam_104122061850', 'cam_037522062165', 'cam_036422060215']
        # self.camera = ['cam_f0461559']
        self.task_file = []
        self.keywords = ["pick"]
        self.selected_task_dict = {kw: [] for kw in self.keywords}
        self.pipeline()


    def pipeline(self):
        self.get_selected_task_ids()
        for camera in self.camera:
            for action in self.keywords:
                self._get_all_task_dirs(action)
                self.make_images_from_videos(action, camera)
                # self.generate_proprioceptive_data(action, camera)


        
    def get_selected_task_ids(self):
        task_desc_path = r'D:\Github\ICRA_25_workshop\cleaned_task_descriptions.json'
        # Open the JSON file and load its contents
        with open(task_desc_path, 'r') as f:
            data = json.load(f)

        # Populate the dictionary
        for key, value in data.items():
            for kw in self.keywords:
                if kw in value.lower():
                    self.selected_task_dict[kw].append(key)

    def _get_all_task_dirs(self, action):
        """Get all task directories in the dataset"""
        task_file = []
        for item in os.listdir(self.data_path):
            if item.startswith('task_') and not item.endswith('_human') and not item.endswith('_2'):
                if item.split('_')[1] in self.selected_task_dict[action]:
                        task_file.append(item)
        self.task_file = task_file

    def is_similar_frame(prev_frame, current_frame, threshold=0.95):
        grayA = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
        grayB = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)
        score, _ = ssim(grayA, grayB, full=True)
        return score > threshold  # If similarity is high, consider as redundant


    def _load_video_frames(self, video_path, depth_path, timestamps, output_dir, csv_file, frame_cut, frame_step=2):
        try:
            cap1 = cv2.VideoCapture(video_path)
            total_frames = int(cap1.get(cv2.CAP_PROP_FRAME_COUNT))
            frame_number = 0
            saved_frames = 0
            prev_frame = None
            timestamps_list = [] 
            
            # Check if file exists to add headers if it's a new file
            file_exists = os.path.isfile(csv_file)
            
            # Prepare data to write
            # task_id = split_task[1]
            # timestamps = timestamp_new['color']
            length = len(timestamps)
            rgb_path_out = os.path.join(output_dir, 'rgb')
            depth_path_out = os.path.join(output_dir, 'depth')
            os.makedirs(rgb_path_out, exist_ok=True)
            os.makedirs(depth_path_out, exist_ok=True)

            while cap1.isOpened():
                ret, frame = cap1.read()
                if not ret:
                    break
                

                # Extract every 'frame_step' frame to reduce storage
                if (frame_number % frame_step == 0) and (frame_number >= frame_step*frame_cut) and (frame_number < length):
                    cv2.imwrite(os.path.join(output_dir, 'rgb', f"{timestamps[frame_number]}.jpg"), frame, [cv2.IMWRITE_JPEG_QUALITY, 85])
                    saved_frames += 1
                    timestamps_list.append(timestamps[frame_number])
                frame_number += 1

                if frame_number >= total_frames:
                    break
            cap1.release()


            cap2 = cv2.VideoCapture(depth_path)
            total_frames = int(cap2.get(cv2.CAP_PROP_FRAME_COUNT))
            frame_number = 0
            saved_frames = 0
            timestamps_list = [] 

            while cap2.isOpened():
                ret, frame = cap2.read()
                if not ret:
                    break

                # Extract every 'frame_step' frame to reduce storage
                if (frame_number % frame_step == 0) and (frame_number >= frame_step*frame_cut) and (frame_number < length):
                    cv2.imwrite(os.path.join(output_dir, 'depth', f"{timestamps[frame_number]}.jpg"), frame, [cv2.IMWRITE_JPEG_QUALITY, 85])
                    saved_frames += 1
                    timestamps_list.append(timestamps[frame_number])
                frame_number += 1

                if frame_number >= total_frames:
                    break
            cap2.release()

            # Write to CSV
            with open(csv_file, "a", newline="") as f:
                writer = csv.writer(f)

                # Write header if new file
                if not file_exists:
                    writer.writerow(["Timestamp", "ID"])

                # Write each timestamp with the corresponding task ID
                for timestamp in timestamps_list:
                    writer.writerow([timestamp, ""])

        except Exception as e:
            print(f"Error loading video: {video_path}, {e}")


    def make_images_from_videos(self, action, camera):
        for task in self.task_file:
            split_task = task.split('_')
            user_scene = '_'.join(split_task[2:6])
            dir_path_robot = os.path.join(self.data_path, task, camera)
            depth_path_robot = os.path.join(self.depth_path, task, camera)
            # Check if the directory exists before proceeding
            if os.path.exists(dir_path_robot):
                # outputdir_robot = os.path.join(self.root_dir_output, 'robot', camera, action, split_task[1], user_scene, 'frames')
                outputdir_robot = os.path.join(self.root_dir_output, 'robot', camera, action, split_task[1], user_scene)
                os.makedirs(outputdir_robot, exist_ok=True)
                video_path_rgb_robot = os.path.join(dir_path_robot, 'color.mp4')
                video_depth_path_robot = os.path.join(depth_path_robot, 'depth.mp4')
                # timestamp_path = os.path.join(dir_path_robot, 'timestamps.npy')
                timestamp_path = os.path.join(self.data_path, task, 'cam_104122061850', 'timestamps.npy')
                timestamps = np.load(timestamp_path, allow_pickle=True)
                timestamp_new = timestamps.item() 

                csv_filename_robot = os.path.join(self.root_dir_output, 'robot', camera, action, "data.csv")
                # Create directory for output
                os.makedirs(os.path.dirname(csv_filename_robot), exist_ok=True)

                self._load_video_frames(video_path_rgb_robot, video_depth_path_robot, timestamp_new['color'], outputdir_robot, csv_filename_robot, frame_cut=13)

            dir_path_human = os.path.join(self.data_path, task+'_human', camera)
            depth_path_human = os.path.join(self.depth_path, task+'_human', camera)
            # Check if the directory exists before proceeding
            if os.path.exists(dir_path_human):
                # outputdir_human = os.path.join(self.root_dir_output, 'human', camera, action, split_task[1], user_scene, 'frames')
                outputdir_human = os.path.join(self.root_dir_output, 'human', camera, action, split_task[1], user_scene)
                os.makedirs(outputdir_human, exist_ok=True)
                video_path_rgb_human = os.path.join(dir_path_human, 'color.mp4')
                video_depth_path_human = os.path.join(depth_path_human, 'depth.mp4')
                # timestamp_path = os.path.join(dir_path_human, 'timestamps.npy')
                timestamp_path = os.path.join(self.data_path, task+'_human', 'cam_104122061850', 'timestamps.npy')
                timestamps = np.load(timestamp_path, allow_pickle=True)
                timestamp_new = timestamps.item()

                csv_filename_human = os.path.join(self.root_dir_output, 'human', camera, action, "data.csv")
                # Create directory for output
                os.makedirs(os.path.dirname(csv_filename_human), exist_ok=True)

                self._load_video_frames(video_path_rgb_human, video_depth_path_human, timestamp_new['color'], outputdir_human, csv_filename_human, frame_cut=18)

    def load_save_proprio_data(self, data_dir, cam, outdir):
        camera = cam.split('_')[1]
        
        tcp_path = os.path.join(data_dir, 'tcp.npy')
        tcp = np.load(tcp_path, allow_pickle=True)
        tcp = tcp.item()
        tcp_camera = tcp[camera]
        np.save(os.path.join(outdir, 'tcp.npy'), tcp_camera) 

        gripper_path = os.path.join(data_dir, 'gripper.npy')
        gripper = np.load(gripper_path, allow_pickle=True)
        gripper = gripper.item()
        gripper_camera = gripper[camera]
        np.save(os.path.join(outdir, 'gripper.npy'), gripper_camera)

        force_torque_path = os.path.join(data_dir, 'force_torque.npy')
        force_torque = np.load(force_torque_path, allow_pickle=True)
        force_torque = force_torque.item()
        force_torque_camera = force_torque[camera]
        np.save(os.path.join(outdir, 'force_torque.npy'), force_torque_camera)

    def generate_proprioceptive_data(self, action, camera):
        for task in self.task_file:
            split_task = task.split('_')
            user_scene = '_'.join(split_task[2:6])
            dir_path_robot = os.path.join(self.data_path, task, 'transformed')
            outputdir_robot = os.path.join(self.root_dir_output, 'robot', camera, action, split_task[1], user_scene)
            os.makedirs(outputdir_robot, exist_ok=True)

            self.load_save_proprio_data(dir_path_robot, camera, outputdir_robot)



In [3]:
dataset = RH20TDataset(r"E:\Neuro_Sym_AI\RH20T_cfg2\RH20T_cfg2")